In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
from response.feature import Feature

In [3]:
path = "../../corpus/hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [4]:
def make_Xy(convs):
    X = []
    y = []
    for conv in convs:
        for i, ut in enumerate( conv ):
            # if ut.is_utt_level_error():
            #     continue
            X.append(ut.utt)
            if ut.is_exist_type():
                y.append(1)
            else:
                y.append(0)
    return X, y

In [5]:
X, y = make_Xy(convs)

In [6]:
len(y)

4200

In [7]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5, stratify=y)

In [8]:
F = Feature()
F_path = "../X_y_data/response2/"
F_name = "Classify_F3.pickle"
featureM = DataManager(F_path)
F.make_features(X_train_str)
featureM.save_data(F_name, F)

success save : ../X_y_data/response2/Classify_F2.pickle


In [9]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( tqdm( X_train_str) ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( tqdm(X_test_str) ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

  0%|          | 0/2940 [00:00<?, ?it/s]

67919


100%|██████████| 1260/1260 [04:57<00:00,  4.23it/s]


In [10]:
from sklearn import svm
# clf = AdaBoostClassifier()
clf = svm.SVC(kernel='rbf', gamma =0.0001, C=1000)

In [11]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [12]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[925  42]
 [ 45 248]]
accuracy =  0.930952380952381
precision =  0.8551724137931035
recall =  0.8464163822525598
f1 score =  0.8507718696397942


- 発話レベルエラーを除く

        confusion matrix = 
        [[817  38]
        [ 27 263]]
        accuracy =  0.9432314410480349
        precision =  0.8737541528239202
        recall =  0.906896551724138
        f1 score =  0.8900169204737732

- 発話レベルを含む

        confusion matrix = 
        [[925  42]
        [ 45 248]]
        accuracy =  0.930952380952381
        precision =  0.8551724137931035
        recall =  0.8464163822525598
        f1 score =  0.8507718696397942

In [13]:
model_path = "../models/response2/"
# M2: 発話レベルエラーを除く
# M3: 発話レベルエラーを含む
model_name = "Classify_M3.pickle"
modelM = DataManager(model_path)
modelM.save_data(model_name, clf)

success save : ../models/response2/Classify_M3.pickle


In [18]:
for y_p, y_t,  x_s in zip(y_pred, y_test, X_test_str):
    # print("{0} : {1}".format(y_p, x_s))
    if y_t==0 and y_p==1:
        print("未検出: {0} : {1}".format(y_p, x_s))

未検出: 1 : どんな食べ物にも合いますね
未検出: 1 : 何ですかそれは、それを聞いて、食べる気が失せました。
未検出: 1 : こんにちは。音楽はどんなのを聴かれていますか？
未検出: 1 : 私も清水寺に行きたいなあと考えてます。
未検出: 1 : TBSのドラマのずっとあなたが好きだったでは、矢部浩之というマザコン男性が、1992年の社会現象となりましたね。
未検出: 1 : そうだね．やっぱりみかんが好きなのかな？
未検出: 1 : そうなんですか？
未検出: 1 : 大丈夫ですか
未検出: 1 : ん〜〜〜〜〜友達と遊ぶことかな？
未検出: 1 : どちらかというとざわちんかな。　モノマネメイクしてみたくならない？
未検出: 1 : 夏は電気代がかかりますよね？
未検出: 1 : 何処にあるお店ですか？ベースコートもトップコートも塗ると持ちが違いますよ。
未検出: 1 : 日本テレビに来春アナウンサーとして入社予定だった南原清隆さんが内定取消をされたニュースをどう思いますか？ 。
未検出: 1 : 私もどちらかというとインドア派で家でのんびり映画とか見てるのが好きなんですが、１３さんは、家でどんな風にのんびり過ごすのが好きですか？
未検出: 1 : やりがいがありそうなお仕事ですね。ちなみに好きな動物は何ですか？
未検出: 1 : 楽しい事をしていると時間を忘れますよね？
未検出: 1 : 恩納村のフォレストアドベンチャーのような娯楽施設での体験を好んでする方ですか？ 。
未検出: 1 : 1000円ぐらいかな？
未検出: 1 : そうですか？
未検出: 1 : 私もどちらかというとインドア派で家でのんびり映画とか見てるのが好きなんですが、１３さんは、家でどんな風にのんびり過ごすのが好きですか？
未検出: 1 : え〜欲しいの？わたしは要らないな
未検出: 1 : どうして．嫌いですか？
未検出: 1 : 神戸にある、GREEN CAFE STYLE・ 茶乃逢やニトリ、阪神タイガースというチーズケーキのお店で販売されているチーズケーキを食べたことはありますか？ 。
未検出: 1 : 熱中症に気をつけてたいですか？
未検出: 1 : 私は名古屋の方が関心ありますね
未検出: 1 : 何年生ですか？
未検出: 1 : こんにちは！東京ディズニーランドと国